<a href="https://colab.research.google.com/github/richardachen/Fireproof/blob/main/Downloading%20GeoTIFF/LFMC_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes to keep in mind for future

https://developers.google.com/earth-engine/guides/image_upload

In [1]:
# https://github.com/kkraoj/lfmc_from_sar

# -*- coding: utf-8 -*-
"""
Created on Thu Mar 19 11:14:09 2020
@author: kkrao
"""


# -*- coding: utf-8 -*-
"""
Created on Tue Jul 31 08:30:32 2018
@author: kkrao
"""

# Import Packages

import ee
from ee import batch
from pandas.tseries.offsets import DateOffset
import pandas as pd

In [2]:
## Initialize (a ee python thing)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0w75kcs8pgDLVkC_BelakgnGV6flNrLClMugapC_sEY&tc=NhladHuPwV7-MRLqfnflneTdZPrMNaOFIFTl5yW4q88&cc=QVzJCAJOZCA0Nw2HGZTYfCM45Jq7mh7wfCHxMUBgu3w

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhh3xwJlLOXheZevzSJ1RgOseuHnU_hCdah7NzS8nWpUvhSOt2lNek

Successfully saved authorization token.


In [9]:
#%%
# Woolsey fire dates
start_date = '2018-10-05'
end_date = '2018-12-31'
folder_name = 'lfmc_folder' # folder name in Google drive where files should be created
scale = 250 #pixel size in meters. lower pixels will consumer more memory and will take longer to download. 

In [10]:
# Location of interest: Greater Los Angeles Area
# https://www.google.com/maps/place/Greater+Los+Angeles,+CA/@34.2817777,-117.9267511,8.27z/data=!4m5!3m4!1s0x80c4ae9046507dd1:0x5798e2322b786355!8m2!3d33.8726016!4d-118.2848067?hl=en&authuser=1

# Woolsey fire locations
la_lon = -117.9267511
la_lat = 34.2817777
la_poi = ee.Geometry.Point(la_lon, la_lat)
la = la_poi.buffer(1e5) #1e5 meters

In [11]:
#%%#### create strings for start and end dates

collection = ee.ImageCollection('users/kkraoj/lfm-mapper/lfmc_col_12_oct_2020').filterDate(start_date,end_date)

# crs --> coordinate reference system

crs = ee.Image(collection.first()).projection();

def resample(image):
  """
  Resample to new resolution
  """
  image = image.resample('bilinear').reproject(crs=crs,scale=scale)
  return image

if scale!=250: #if user requested resolution is different than native resolution (250m), resample to new resolution
    collection = collection.map(resample)

In [12]:
n = collection.size().getInfo() # number of images to download
colList = collection.toList(n)
  
for i in range(n):
    image = ee.Image(colList.get(i));
    id = image.id().getInfo() or 'image_'+i.toString();

    out = batch.Export.image.toDrive(
      image=image,
      folder=folder_name,
      description=id,
      region = la,
      scale=scale,
      crs=crs,
      fileFormat='GeoTIFF',
      maxPixels=1e11
    );
    batch.Task.start(out)
## process the image

out.status()
print("process sent to cloud")

process sent to cloud
